# Layout generator for Simba Report

@Raphael Dayan -- analytics@mercadopago.com.br

https://asspaweb.pgr.mpf.gov.br/site/index.php/sistemas/sigilo-bancario-simba

In [ ]:
#!pip install http://pypi.ml.com/api/package/melitk-analytics/melitk.analytics-1.4.0.tar.gz
#!pip install -i http://pypi.ml.com/simple/melitk-analytics/melitk.analytics-1.4.0.tar.gz
#!pip install openpyxl
#!pip install -i http://pypi.ml.com/simple/ melitk-fda --trusted-host pypi.ml.com
#!pip install -i http://pypi.ml.com/simple/ melitk-analytics --trusted-host pypi.ml.com

from datetime import date
import datetime as dt
import os
import pickle
import pandas as pd
import s3fs
import numpy as np
import csv
import openpyxl
import time
from IPython.display import clear_output

from melitk.analytics.connectors.core.authentication import Authentication
from melitk.analytics.connectors.teradata import ConnTeradata
from melitk.fda import workspace

teradata_user = os.environ['SECRET_TERADATA_USER']
teradata_pass = os.environ['SECRET_TERADATA_PASS']
tera = ConnTeradata(teradata_user, teradata_pass, auth_method=Authentication.APP)
print('Connection has started at', dt.datetime.now())

In [ ]:
# Import CSV File for newly created Oficios
print('Import has been started at', dt.datetime.now())
csv_file = '/alloc/data/fury_oficios-quebra-sigilo/notebooks/csv_r.csv'
csv_oficios = pd.read_csv(csv_file, engine='python', sep = ';', parse_dates=['DATA_OFICIO', 'RANGE_MIN', 'RANGE_MAX', 'CREATED_AT'], encoding = 'iso-8859-1',  converters={'DOC_NUMBER': lambda x: str(x),
                                                                                                                                                                          'NUMERO_PROCESSO': lambda x: str(x)})
csv_oficios = csv_oficios.astype(str)
print('File has been processed and can be used at', dt.datetime.now())
#csv_oficios['DOC_NUMBER'].str.len().drop_duplicates() # to check len
#csv_oficios['CREATED_AT'].drop_duplicates() 
#csv_oficios[csv_oficios['DOC_NUMBER'].str.len() == 13]
#csv_oficios[~csv_oficios['DOC_NUMBER'].str.isdigit()]
#csv_oficios['DATA_OFICIO'].drop_duplicates() 
#csv_oficios['RANGE_MIN'].drop_duplicates() 
#csv_oficios['RANGE_MAX'].drop_duplicates() 

In [ ]:
# # Create excel table if needed. Otherwise use insert below
# counter = 1 
# with open("./Queries/create_csv_table.sql", "r") as f:
#    create_csv_table = f.read().split(";")[:-1]
#   total_queries = str(len(create_csv_table))
# print('Total queries to run: ' + total_queries + '\n')

#     for query in create_csv_table:
#         print('Running query #', counter, 'out of ' + total_queries + query[:50], '...')
#         start_query = time.time()
#         print('Querying process started at', dt.datetime.now())
#         tera.execute(query)
#         end_query = time.time()
#         print('Querying took %1.2f' %(end_query-start_query), 'seconds to run.')
#         print('Querying process ended at', dt.datetime.now(), '\n')
#         counter +=1
#         clear_output()

# Insert new rows into documents to be reported log
print('Insert has been started at', dt.datetime.now())
start_query = time.time()
df = csv_oficios
data = df.values.tolist()
for i in range(0,len(df),1000):
    insert_query = ''
    for j in range(1000):
        if (i + j) < len(df):            
            insert_query += """INSERT MP_MPB.DOC_INPUT values ('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}');""".format(*data[i+j])
    tera.execute(insert_query)
end_query = time.time()
print('Insert took %1.2f' %(end_query-start_query), 'seconds to run.')
print('File has been processed and can be used at', dt.datetime.now())

In [ ]:
# #Create matched doc info to be used as parameter to search ranges if needed. Otherwise use insert below
# with open("./Queries/doc_info_match.sql", "r") as f:
#     doc_info_match_response = f.read().split(";")[:-1]
    
#     for query in doc_info_match_response:
#         tera.execute(query)

# Insert doc info match
with open("./Queries/insert_doc_info_match.sql", "r") as f:
    insert_doc_info_match_response = f.read().split(";")[:-1]
    
    for query in insert_doc_info_match_response:
        tera.execute(query)
print('Has run at', dt.datetime.now())

In [ ]:
# Get Account Movement History Log
with open("./Queries/account_movement.sql", "r") as f:
    account_movement_response = f.read().split(";")[:-1]
    
    for query in account_movement_response:
        tera.execute(query)
print('Has run at', dt.datetime.now())

In [ ]:
#query1=pd.DataFrame(tera.execute_response('select * from MP_MPB.ACC_MOV_HISTORY_TEMP sample 10'))
#query1=pd.DataFrame(tera.execute_response('SELECT * FROM MP_MPB.DOC_INPUT'))
#query1
#query1=pd.DataFrame(tera.execute_response('SELECT DISTINCT CUS_CUST_ID FROM MP_MPB.DOC_INPUT_MATCH WHERE CREATED_AT = (SELECT MAX(CREATED_AT) FROM MP_MPB.DOC_INPUT)'))

In [ ]:
# Get Withdrawal History Log 
with open("./Queries/withdrawal_movement.sql", "r") as f:
    account_movement_response = f.read().split(";")[:-1]
    
    for query in account_movement_response:
        tera.execute(query)
print('Has run at', dt.datetime.now())

In [ ]:
# Join Movements + Withdrawal
with open("./Queries/joined_mov_history.sql", "r") as f:
    joined_movement_response = f.read().split(";")[:-1]
    
    for query in joined_movement_response:
        tera.execute(query)
print('Has run at', dt.datetime.now())

In [ ]:
# Grab docs with their ranges to report
with open("./Queries/doc_info_ranges.sql", "r") as f:
    custs_response = f.read().split(";")[:-1]
    
    for query in custs_response:
        custs = tera.execute_response(query)
        custs = pd.DataFrame(custs)
        custs_range = custs[['COMPUTADOR_DESTINO', 'NUMERO_CASO', 'CUS_CUST_ID', 'RANGE_MIN', 'RANGE_MAX', 'CREATED_AT']]
        custs_range['OFICIO_NUMBER'] = custs[['COMPUTADOR_DESTINO', 'NUMERO_CASO']].apply(lambda x: '-'.join(x), axis=1)
        custs_range = custs_range.astype({"CUS_CUST_ID": int})
        custs_range = custs_range.astype({"OFICIO_NUMBER": str})
        custs_range = custs_range[['OFICIO_NUMBER',  'CREATED_AT', 'CUS_CUST_ID', 'RANGE_MIN', 'RANGE_MAX']]
        #custs_range = custs_range.head(50)
print('Has run at', dt.datetime.now())

In [ ]:
# Create table with each customer suit-tailored range 
with open("./Queries/create_movement_table.sql", "r") as f:
    mov_response = f.read().split(";")[:-1]
    
    for query in mov_response:
        tera.execute(query)
print('Has run at', dt.datetime.now())

In [ ]:
# Insert into table each Oficios Number and customer ranges
cust_ranges_list = []
for param in custs_range.values.tolist():
    cust_ranges_list.append(
"""    
INSERT INTO MP_MPB.ACC_MOV_HISTORY_REPORT
SELECT
      CUS_CUST_ID,
      CUS_CUST_ID_SEL,
      CUS_CUST_ID_BUY,
      MOV_MOVE_ID,
      MOV_CREATED_DT,
      MOV_DETAIL,
      MOV_TYPE_ID,
      PAY_PAYMENT_ID,
      PAY_REASON_ID,
      PAY_OPERATION_TYPE_ID,
      MOV_FINANCIAL_ENTITY_ID,
      WIT_WITHDRAW_ID,
      MOV_AMOUNT,
      ACCOUNT_TYPE,
      WIT_BANK_ACC_NUMBER,
      WITHDRAWAL_STATUS,
      RECEIVING_ACCOUNT_HOLDER_NAME,
      RECEIVING_ACCOUNT_HOLDER_DOC_NUMBER,
      BANK_ID,
      WIT_BANK_ACC_BRANCH_ID,
      RECEIVING_BANK_NAME,
      PAYER_BANK_NAME,
      WITHDRAWAL_METHOD,
      WITHDRAWAL_AMOUNT,
      CAST('{oficio_number}' AS VARCHAR(400)) AS OFICIO_NUMBER,
      CAST('{oficio_created_at}' AS VARCHAR(400)) AS OFICIO_CREATED_AT
FROM MP_MPB.USER_MOV_HISTORY
WHERE 1=1
      AND CUS_CUST_ID = {cust_id}
      AND MOV_CREATED_DT BETWEEN '{range_min}' AND '{range_max}'
""".format(oficio_number = param[0], oficio_created_at = param[1], cust_id = param[2], range_min = param[3], range_max = param[4]) + ";"
)
    
counter = 1
total_queries = str(len(cust_ranges_list))
print('Total queries to run: ' + total_queries + '\n')

for query in cust_ranges_list:
    print('Running query #', counter, 'out of ' + total_queries + query[:50], '...')
    start_query = time.time()
    print('Querying process started at', dt.datetime.now())
    tera.execute(query)
    end_query = time.time()
    print('Querying took %1.2f' %(end_query-start_query), 'seconds to run.')
    print('Querying process ended at', dt.datetime.now(), '\n')
    counter +=1
    clear_output()
    
print('Has run at', dt.datetime.now())

In [ ]:
# Create customer info table
with open("./Queries/cus_info.sql", "r") as f:
    customer_response = f.read().split(";")[:-1]
    
    for query in customer_response:
        tera.execute(query)
print('Has run at', dt.datetime.now())

In [ ]:
# # # # Create report log table if needed. Otherwise use insert below
# with open("./Queries/create_oficios_table.sql", "r") as f:
#     oficios_create_response = f.read().split(";")[:-1]
    
#     for query in oficios_create_response:
#         tera.execute(query)

# Insert info @log table
with open("./Queries/insert_oficios_table.sql", "r") as f:
    oficios_insert_response = f.read().split(";")[:-1]
    
    for query in oficios_insert_response:
        tera.execute(query)
print('Has run at', dt.datetime.now())

In [ ]:
query

In [ ]:
# Log table to start data cleansing and layout development
with open("./Queries/oficios_db.sql", "r") as f:
    oficios_db_response = f.read().split(";")[:-1]
    
    for query in oficios_db_response:
        oficios_tbl = tera.execute_response(query)
        oficios_df = pd.DataFrame(oficios_tbl)
        #oficios_df = oficios_df.head(50)
        #oficios_df = oficios_df[oficios_df['OFICIO_NUMBER'] == "059-PCRO-000028-45"]
        #of = ["093"O-000261-69","001-MPF-004416-70","061-PCCE-000078-00","001-MPF-004469-81","018-PCSP-000554-55","001-MPF-004350-09"]
        #oficios_df = oficios_df[oficios_df.OFICIO_NUMBER.isin(of)]
print(oficios_df.shape, dt.datetime.now())

In [ ]:
# Oficios information to send emails + csvs
with open("./Queries/oficios_info.sql", "r") as f:
    oficios_info_response = f.read().split(";")[:-1]
    
    for query in oficios_info_response:
        oficios_info_tbl = tera.execute_response(query)
        oficios_info_df = pd.DataFrame(oficios_info_tbl)
        #oficios_info_df =  oficios_info_df[oficios_info_df['OFICIO_NUMBER'] == "059-PCRO-000028-45"]
        #oficios_info_df = oficios_info_df[oficios_info_df.OFICIO_NUMBER.isin(of)]
print(oficios_info_df.shape, dt.datetime.now())

In [ ]:
# Doc number range to fill investigados.txt
with open("./Queries/investigados_doc_range.sql", "r") as f:
    doc_info_response = f.read().split(";")[:-1]
    
    for query in doc_info_response:
        doc_info_tbl = tera.execute_response(query)
        doc_info_df = pd.DataFrame(doc_info_tbl)
        #doc_info_df = doc_info_df[doc_info_df['OFICIO_NUMBER'] == "059-PCRO-000028-45"]
        #doc_info_df = doc_info_df[doc_info_df.OFICIO_NUMBER.isin(of)]
        doc_info_df.rename(columns = {'DOC_NUMBER': 'CUS_CUST_DOC_NUMBER'}, inplace = True)
print(doc_info_df.shape, dt.datetime.now())

In [ ]:
agencias['HAD_REPORT'] 

In [ ]:
# Starting generation of 'AGENCIAS' layout
agencias = oficios_df[['OFICIO_NUMBER', 'BANK_ID', 'WIT_BANK_ACC_BRANCH_ID', 'RECEIVING_BANK_NAME', 'CUS_RU_SINCE_DT_BUY', 'HAD_REPORT']].drop_duplicates()
agencias['OFICIO_NUMBER'] = agencias['OFICIO_NUMBER']
agencias['HAD_REPORT'] = agencias['HAD_REPORT']
#agencias['NUMERO_BANCO'] = agencias['BANK_ID'].fillna('033')
agencias['NUMERO_BANCO'] = "323"
#agencias['NUMERO_AGENCIA'] = agencias['WIT_BANK_ACC_BRANCH_ID'].fillna('2167')
agencias['NUMERO_AGENCIA'] = "0001"
#agencias['NOME_AGENCIA'] = agencias['RECEIVING_BANK_NAME'].fillna('Mercado Pago')
agencias['NOME_AGENCIA'] = "Mercado Pago"
#agencias['ENDERECO_LOGRADOURO'] = np.where(agencias['NUMERO_BANCO'] == "033", 'AVENIDA DAS NACOES UNIDAS, 3003', 'Nao ha endereco logradouro')
agencias['ENDERECO_LOGRADOURO'] = "AVENIDA DAS NACOES UNIDAS, 3003"
#agencias['ENDERECO_CIDADE'] = np.where(agencias['NUMERO_BANCO'] == "033", 'OSASCO', 'Nao ha endereco cidade')
agencias['ENDERECO_CIDADE'] = "OSASCO"
#agencias['ENDERECO_UF'] = np.where(agencias['NUMERO_BANCO'] == "033", 'SP', 'XX')
agencias['ENDERECO_UF'] = "SP"
#agencias['ENDERECO_PAIS'] = np.where(agencias['NUMERO_BANCO'] == "033", 'BRASIL', 'BRASIL')
agencias['ENDERECO_PAIS'] = "BRASIL"
#agencias['ENDERECO_CEP'] = np.where(agencias['NUMERO_BANCO'] == "033", '06233903', '00000000')
agencias['ENDERECO_CEP'] = "06233903"
#agencias['TELEFONE_AGENCIA'] = np.where(agencias['NUMERO_BANCO'] == "033", '1125434155', '0000000000')
agencias['TELEFONE_AGENCIA'] = "1125434155"
agencias['DATA_ABERTURA_AGENCIA'] = np.where(agencias['NUMERO_BANCO'] == "033", '22122008', agencias['CUS_RU_SINCE_DT_BUY'].fillna('00000000')) #13062019
agencias['DATA_ABERTURA_AGENCIA'] = "22122008"
agencias['DATA_FECHAMENTO_AGENCIA'] = ""
agencias = agencias[['HAD_REPORT', 'OFICIO_NUMBER', 'NUMERO_BANCO', 'NUMERO_AGENCIA', 'NOME_AGENCIA', 
                     'ENDERECO_LOGRADOURO', 'ENDERECO_CIDADE',
                     'ENDERECO_UF', 'ENDERECO_PAIS', 'ENDERECO_CEP', 
                     'TELEFONE_AGENCIA', 'DATA_ABERTURA_AGENCIA', 'DATA_FECHAMENTO_AGENCIA']].drop_duplicates()
agencias_df = pd.DataFrame(agencias)

# Correcting 'AGENCIAS' layout
    # NUMERO_BANCO = 3 char
    # NUMERO_AGENCIA = 4 char
    # NOME_AGENCIA = 50 char
    # ENDERECO_LOGRADOURO = 80 char
    # ENDERECO_CIDADE = 40 char
    # ENDERECO_UF = 2 char
    # ENDERECO_PAIS = 40 char
    # ENDERECO_CEP = 8 char
    # TELEFONE_AGENCIA = 20 char
    # DATA_ABERTURA_AGENCIA = 8 char, DDMMYYYY
    # DATA_FECHAMENTO_AGENCIA = 8 char, DDMMYYYY

agencias_df['NUMERO_BANCO'] = agencias_df['NUMERO_BANCO'].str.ljust(3, ' ')
agencias_df['NUMERO_AGENCIA'] = agencias_df['NUMERO_AGENCIA'].str.ljust(4, ' ')
agencias_df['NOME_AGENCIA'] = agencias_df['NOME_AGENCIA'].str.ljust(50, ' ')
agencias_df['ENDERECO_LOGRADOURO'] = agencias_df['ENDERECO_LOGRADOURO'].str.ljust(80, ' ')
agencias_df['ENDERECO_CIDADE'] = agencias_df['ENDERECO_CIDADE'].str.ljust(40, ' ')
agencias_df['ENDERECO_UF'] = agencias_df['ENDERECO_UF'].str.ljust(2, ' ')
agencias_df['ENDERECO_PAIS'] = agencias_df['ENDERECO_PAIS'].str.ljust(40, ' ')
agencias_df['ENDERECO_CEP'] = agencias_df['ENDERECO_CEP'].str.ljust(8, ' ')
agencias_df['TELEFONE_AGENCIA'] = agencias_df['TELEFONE_AGENCIA'].str.ljust(20, ' ')
agencias_df['DATA_ABERTURA_AGENCIA'] = agencias_df['DATA_ABERTURA_AGENCIA'].str.ljust(8, ' ') 
agencias_df['DATA_FECHAMENTO_AGENCIA'] = agencias_df['DATA_FECHAMENTO_AGENCIA'].str.ljust(8, ' ')
agencias_df = agencias_df[agencias_df['NUMERO_BANCO'] == "323"]
# agencias_df['ENDERECO_PAIS'].str.len().drop_duplicates() # to check len

# Generating archive names + rows
column_names = ["REPORT", "NOME_ARQUIVO", "LINHA_ARQUIVO"]
agencias_df_txt = pd.DataFrame(columns = column_names)
agencias_df_txt['REPORT'] = agencias_df['HAD_REPORT']
agencias_df_txt['NOME_ARQUIVO'] = agencias_df['OFICIO_NUMBER']
agencias_df_txt['LINHA_ARQUIVO'] = agencias_df[['NUMERO_BANCO', 'NUMERO_AGENCIA', 'NOME_AGENCIA', 
                     'ENDERECO_LOGRADOURO', 'ENDERECO_CIDADE',
                     'ENDERECO_UF', 'ENDERECO_PAIS', 'ENDERECO_CEP', 
                     'TELEFONE_AGENCIA', 'DATA_ABERTURA_AGENCIA', 'DATA_FECHAMENTO_AGENCIA']].apply(lambda x: '\t'.join(x), axis=1)
agencias_df_txt = agencias_df_txt[agencias_df_txt['REPORT'] == "002"]
agencias_df_csv = agencias_df.merge(oficios_info_df, on='OFICIO_NUMBER', how='left')
agencias_df_csv = agencias_df_csv[agencias_df_csv['HAD_REPORT'] == "002"] 
agencias_df_csv = agencias_df_csv.astype({'HAD_REPORT': str,
                                          'OFICIO_NUMBER': str,
                                          'NUMERO_BANCO': str,
                                          'NUMERO_AGENCIA': str,
                                          'NOME_AGENCIA': str,
                                          'ENDERECO_LOGRADOURO': str,
                                          'ENDERECO_CIDADE': str,
                                          'ENDERECO_UF': str,
                                          'ENDERECO_PAIS': str,
                                          'ENDERECO_CEP': str,
                                          'TELEFONE_AGENCIA': str,
                                          'DATA_ABERTURA_AGENCIA': str,
                                          'DATA_FECHAMENTO_AGENCIA': str,
                                          'CARGO_MAGISTRADO': str,
                                          'DATA_OFICIO': str,
                                          'EMAIL_CASO': str,
                                          'NOME_MAGISTRADO': str,
                                          'NOME_TRIBUNAL': str,
                                          'NUMERO_PROCESSO': str})
print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
# Generating 'AGENCIAS' .txt
for row in agencias_df_txt.groupby('NOME_ARQUIVO'):
    row[1]['LINHA_ARQUIVO'].to_csv('/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/'+str(row[0]).strip('[]').strip("'")+'_AGENCIAS.txt', 
                                      index = False, 
                                      header = False,
                                      quotechar= "",
                                      sep = "\n",
                                      escapechar = '\\',
                                      quoting=csv.QUOTE_NONE)

# Create excel file with sheet = AGENCIAS to be sent @email.
agencias_grouped = agencias_df_csv.groupby('OFICIO_NUMBER')
for ag_titles, ag_rows in agencias_grouped:
    with pd.ExcelWriter('/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/'+str(ag_titles)+'.xlsx') as writer:  
        ag_rows.to_excel(writer, sheet_name='AGENCIAS', index= False, header = True)

# If you want to save it separately as .csv
# agencias_grouped = agencias_df_csv.groupby('OFICIO_NUMBER')
# for titles, rows in agencias_grouped:
#     rows.to_csv('/alloc/data/fury_oficios-quebra-sigilo/notebooks/Arquivos/'+str(titles)+'.csv',
#                                       index = False,
#                                       header= True)
print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
# Starting generation of 'CONTAS' layout
contas = oficios_df[['OFICIO_NUMBER', 'BANK_ID', 'WIT_BANK_ACC_BRANCH_ID', 'WIT_BANK_ACC_NUMBER', 'ACCOUNT_TYPE', 'HAD_REPORT']].drop_duplicates()
contas['OFICIO_NUMBER'] = contas['OFICIO_NUMBER']
contas['HAD_REPORT'] = contas['HAD_REPORT']
#contas['NUMERO_BANCO'] = contas['BANK_ID'].fillna('033')
contas['NUMERO_BANCO'] = "323"
#contas['NUMERO_AGENCIA'] = contas['WIT_BANK_ACC_BRANCH_ID'].fillna('2167')
contas['NUMERO_AGENCIA'] = "0001"
#contas['NUMERO_CONTA'] = contas['WIT_BANK_ACC_NUMBER'].fillna('130010391')
contas['NUMERO_CONTA'] = "10573521"
# contas['TIPO_CONTA'] = np.where(contas['NUMERO_BANCO'] == "323", '4',
#                   np.where(contas['ACCOUNT_TYPE'] == "checking_account", '1',
#                   np.where(contas['ACCOUNT_TYPE'] == "savings_account", '2', '4')))
contas['TIPO_CONTA'] = "4"
#contas['DATA_ABERTURA_CONTA'] = np.where(contas['NUMERO_BANCO'] == "323", '22122008', '')
contas['DATA_ABERTURA_CONTA'] = "22122008"
contas['DATA_ENCERRAMENTO_CONTA'] = ""
contas['MOVIMENTACAO_CONTA'] = '1'
contas = contas[['HAD_REPORT', 'OFICIO_NUMBER', 'NUMERO_BANCO', 'NUMERO_AGENCIA', 
                  'NUMERO_CONTA', 'TIPO_CONTA',
                  'DATA_ABERTURA_CONTA', 'DATA_ENCERRAMENTO_CONTA',
                  'MOVIMENTACAO_CONTA']].drop_duplicates()
contas_df = pd.DataFrame(contas)

# Correcting 'CONTAS' layout
    # NUMERO_BANCO = 3 char
    # NUMERO_AGENCIA = 4 char
    # NUMERO_CONTA = 20 char
    # TIPO_CONTA = 1 char
    # DATA_ABERTURA_CONTA = 8 char, DDMMYYYY
    # DATA_ENCERRAMENTO_CONTA = 8 char, DDMMYYYY
    # MOVIMENTACAO_CONTA =  1 char

contas_df['NUMERO_BANCO'] = contas_df['NUMERO_BANCO'].str.ljust(3, ' ')
contas_df['NUMERO_AGENCIA'] = contas_df['NUMERO_AGENCIA'].str.ljust(4, ' ')
contas_df['NUMERO_CONTA'] = contas_df['NUMERO_CONTA'].str.ljust(20, ' ')
contas_df['TIPO_CONTA'] = contas_df['TIPO_CONTA'].str.ljust(1, ' ')
contas_df['DATA_ABERTURA_CONTA'] = contas_df['DATA_ABERTURA_CONTA'].str.ljust(8, ' ') 
contas_df['DATA_ENCERRAMENTO_CONTA'] = contas_df['DATA_ENCERRAMENTO_CONTA'].str.ljust(8, ' ')
contas_df = contas_df[contas_df['NUMERO_BANCO'] == "323"]
# contas_df['NUMERO_CONTA'].str.len().drop_duplicates() # to check len

# Generating archive names + rows
column_names = ["REPORT", "NOME_ARQUIVO", "LINHA_ARQUIVO"]
contas_df_txt = pd.DataFrame(columns = column_names)
contas_df_txt['REPORT'] = contas_df['HAD_REPORT']
contas_df_txt['NOME_ARQUIVO'] = contas_df['OFICIO_NUMBER']
contas_df_txt['LINHA_ARQUIVO'] = contas_df[['NUMERO_BANCO', 'NUMERO_AGENCIA', 
                  'NUMERO_CONTA', 'TIPO_CONTA',
                  'DATA_ABERTURA_CONTA', 'DATA_ENCERRAMENTO_CONTA',
                  'MOVIMENTACAO_CONTA']].apply(lambda x: '\t'.join(x), axis=1)
contas_df_txt = contas_df_txt[contas_df_txt['REPORT'] == "002"]
contas_df_csv = contas_df.merge(oficios_info_df, on='OFICIO_NUMBER', how='left')
contas_df_csv = contas_df_csv[contas_df_csv['HAD_REPORT'] == "002"]
contas_df_csv = contas_df_csv.astype({'HAD_REPORT': str,
                                          'OFICIO_NUMBER': str,
                                          'NUMERO_BANCO': str,
                                          'NUMERO_AGENCIA': str,
                                          'NUMERO_CONTA': str,
                                          'TIPO_CONTA': str,
                                          'DATA_ABERTURA_CONTA': str,
                                          'DATA_ENCERRAMENTO_CONTA': str,
                                          'MOVIMENTACAO_CONTA': str,
                                          'CARGO_MAGISTRADO': str,
                                          'DATA_OFICIO': str,
                                          'EMAIL_CASO': str,
                                          'NOME_MAGISTRADO': str,
                                          'NOME_TRIBUNAL': str,
                                          'NUMERO_PROCESSO': str})
print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
# Generating 'CONTAS' .txt
for row in contas_df_txt.groupby('NOME_ARQUIVO'):
    row[1]['LINHA_ARQUIVO'].to_csv('/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/'+str(row[0]).strip('[]').strip("'")+'_CONTAS.txt', 
                                      index = False, 
                                      header = False,
                                      quotechar= "",
                                      sep = "\n",
                                      escapechar = '\\',
                                      quoting=csv.QUOTE_NONE)

# Append excel file with sheet = CONTAS to be sent @email.
contas_grouped = contas_df_csv.groupby('OFICIO_NUMBER')
for co_titles, co_rows in contas_grouped:
    with pd.ExcelWriter('/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/'+str(co_titles)+'.xlsx',  mode='a') as writer:  
        co_rows.to_excel(writer, sheet_name='CONTAS', index= False, header = True)

print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
# Starting generation of 'TITULARES' layout
titulares = oficios_df[['OFICIO_NUMBER', 'BANK_ID', 'WIT_BANK_ACC_BRANCH_ID', 'WIT_BANK_ACC_NUMBER', 'ACCOUNT_TYPE', 
                        'CUS_CUST_DOC_NUMBER', 'CUS_FULL_NAME', 'CUS_CUST_DOC_TYPE',
                        'CUS_ADDRESS','CUS_CITY', 'UF',
                        'CUS_ZIP_CODE', 'CUS_RU_SINCE_DT', 'HAD_REPORT']].drop_duplicates()
titulares = titulares[~titulares['CUS_CUST_DOC_NUMBER'].isna()]

cus_full_name = titulares[['CUS_FULL_NAME','CUS_ADDRESS','CUS_CITY', 'UF',
                        'CUS_ZIP_CODE', 'CUS_RU_SINCE_DT']].groupby(titulares['CUS_CUST_DOC_NUMBER']).nth(0).reset_index()

cus_full_name['CUS_FULL_NAME_CORRECTED'] = cus_full_name['CUS_FULL_NAME']
cus_full_name['CUS_ADDRESS_CORRECTED'] = cus_full_name['CUS_ADDRESS']
cus_full_name['CUS_CITY_CORRECTED'] = cus_full_name['CUS_CITY']
cus_full_name['UF_CORRECTED'] = cus_full_name['UF']
cus_full_name['CUS_ZIP_CODE_CORRECTED'] = cus_full_name['CUS_ZIP_CODE']
cus_full_name['CUS_RU_SINCE_DTCORRECTED'] = cus_full_name['CUS_RU_SINCE_DT']
del cus_full_name['CUS_FULL_NAME']
del cus_full_name['CUS_ADDRESS']
del cus_full_name['CUS_CITY']
del cus_full_name['UF']
del cus_full_name['CUS_ZIP_CODE']
del cus_full_name['CUS_RU_SINCE_DT']
titulares = titulares.merge(cus_full_name, on=['CUS_CUST_DOC_NUMBER'], how = 'left')
titulares['OFICIO_NUMBER'] = titulares['OFICIO_NUMBER']
titulares['HAD_REPORT'] = titulares['HAD_REPORT']
#titulares['NUMERO_BANCO'] = titulares['BANK_ID'].fillna('323')
titulares['NUMERO_BANCO'] = "323"
#titulares['NUMERO_AGENCIA'] = titulares['WIT_BANK_ACC_BRANCH_ID'].fillna('0001')
titulares['NUMERO_AGENCIA'] = "0001"
#titulares['NUMERO_CONTA'] = titulares['WIT_BANK_ACC_NUMBER'].fillna('00000000000000000000')
titulares['NUMERO_CONTA'] = "10573521" 
# titulares['TIPO_CONTA'] = np.where(titulares['NUMERO_BANCO'] == "323", '4',
#                    np.where(titulares['ACCOUNT_TYPE'] == "checking_account", '1',
#                    np.where(titulares['ACCOUNT_TYPE'] == "savings_account", '2', '4')))
titulares['TIPO_CONTA'] = "4"
a = titulares[['CUS_CUST_DOC_NUMBER']].groupby(titulares['OFICIO_NUMBER']).nth(0).reset_index()
a['flag'] = "1"
titulares = titulares.merge(a, on= ['OFICIO_NUMBER', 'CUS_CUST_DOC_NUMBER'], how='left')
titulares['TIPO_TITULAR'] = np.where(titulares['flag'] == "1", "T", "O")
titulares['PESSOA_INVESTIGADA'] = np.where(titulares['HAD_REPORT'] == "001", "0", "1")
titulares['TIPO_PESSOA_TITULAR'] = np.where(titulares['CUS_CUST_DOC_TYPE'].isna(), 
                        np.where(titulares['CUS_CUST_DOC_NUMBER'].fillna('').apply(len) == 11, '1', '2')
                                 ,np.where(titulares['CUS_CUST_DOC_TYPE'] == "CPF", "1", "2"))
titulares['CPF_CNPJ_TITULAR'] = titulares['CUS_CUST_DOC_NUMBER'].fillna('')
titulares['NOME_TITULAR'] = titulares['CUS_FULL_NAME_CORRECTED'].fillna('Mercado Pago')
titulares['NOME_DOC_IDENTIFICACAO'] = "Nao ha documento extra" # Preenchendo para testar
titulares['NUMERO_DOC_IDENTIFICACAO'] = "00000000000000000000" # Preenchendo para testar
titulares['ENDERECO_LOGRADOURO'] = titulares['CUS_ADDRESS_CORRECTED'].fillna('AVENIDA DAS NACOES UNIDAS, 3003')
titulares['ENDERECO_CIDADE'] = titulares['CUS_CITY_CORRECTED'].fillna('SAO PAULO')
titulares['ENDERECO_UF'] = titulares['UF_CORRECTED'].fillna('SP')
titulares['ENDERECO_PAIS'] = "BRASIL"
titulares['ENDERECO_CEP'] = titulares['CUS_ZIP_CODE_CORRECTED'].fillna('06233903')
titulares['TELEFONE_PESSOA'] = "00000000000" # Preenchendo para testar
titulares['VALOR_RENDA'] = "00000000000000" # Preenchendo para testar
titulares['DATA_ATUALIZACAO_RENDA'] = '22122008' # titulares['CUS_RU_SINCE_DT_BUY'].fillna('') # Preenchendo para testar
#titulares['DATA_INICIO_RELACIONAMENTO_CONTA'] = titulares['CUS_RU_SINCE_DT_BUY'].fillna('')
#titulares['DATA_INICIO_RELACIONAMENTO_CONTA'] = np.where(titulares['NUMERO_BANCO'] == "323", '13062019',
#                                                         titulares['CUS_RU_SINCE_DT_BUY'].fillna(''))
titulares['DATA_INICIO_RELACIONAMENTO_CONTA'] = "22122008"
titulares['DATA_FIM_RELACIONAMENTO_CONTA'] = ""

titulares = titulares[['HAD_REPORT', 'OFICIO_NUMBER', 'NUMERO_BANCO', 'NUMERO_AGENCIA', 
                  'NUMERO_CONTA', 'TIPO_CONTA',
                  'TIPO_TITULAR', 'PESSOA_INVESTIGADA', 'TIPO_PESSOA_TITULAR',
                  'CPF_CNPJ_TITULAR', 'NOME_TITULAR', 'NOME_DOC_IDENTIFICACAO',
                 'NUMERO_DOC_IDENTIFICACAO', 'ENDERECO_LOGRADOURO', 'ENDERECO_CIDADE',
                'ENDERECO_UF', 'ENDERECO_PAIS', 'ENDERECO_CEP', 'TELEFONE_PESSOA',
                'VALOR_RENDA', 'DATA_ATUALIZACAO_RENDA', 'DATA_INICIO_RELACIONAMENTO_CONTA',
                 'DATA_FIM_RELACIONAMENTO_CONTA']].drop_duplicates()
titulares_df = pd.DataFrame(titulares)

# Correcting 'TITULARES' layout
    # NUMERO_BANCO = 3 char
    # NUMERO_AGENCIA = 4 char
    # NUMERO_CONTA = 20 char
    # TIPO_CONTA = 1 char
    # TIPO_TITULAR = 1 char
    # PESSOA_INVESTIGADA = 1 char
    # TIPO_PESSOA_TITULAR = 1 char
    # CPF_CNPJ_TITULAR = 14 char
    # NOME_TITULAR = 80 char
    # NOME_DOC_IDENTIFICACAO = 50 char
    # NUMERO_DOC_IDENTIFICACAO = 20 char
    # ENDERECO_LOGRADOURO = 80 char
    # ENDERECO_CIDADE = 40 char
    # ENDERECO_UF = 2 char
    # ENDERECO_PAIS = 40 char
    # ENDERECO_CEP = 8 char
    # TELEFONE_PESSOA = 20 char
    # VALOR_RENDA = 14 char
    # DATA_ATUALIZACAO_RENDA = 8 char, DDMMYYY
    # DATA_INICIO_RELACIONAMENTO_CONTA = 8 char, DDMMYYY
    # DATA_FIM_RELACIONAMENTO_CONTA 8 char, DDMMYYY


titulares_df['NUMERO_BANCO'] = titulares_df['NUMERO_BANCO'].str.ljust(3, ' ')
titulares_df['NUMERO_AGENCIA'] = titulares_df['NUMERO_AGENCIA'].str.ljust(4, ' ')
titulares_df['NUMERO_CONTA'] = titulares_df['NUMERO_CONTA'].str.ljust(20, ' ')
titulares_df['TIPO_CONTA'] = titulares_df['TIPO_CONTA'].str.ljust(1, ' ')
titulares_df['TIPO_TITULAR'] = titulares_df['TIPO_TITULAR'].str.ljust(1, ' ')
titulares_df['PESSOA_INVESTIGADA'] = titulares_df['PESSOA_INVESTIGADA'].str.ljust(1, ' ')
titulares_df['TIPO_PESSOA_TITULAR'] = titulares_df['TIPO_PESSOA_TITULAR'].str.ljust(1, ' ')
titulares_df['CPF_CNPJ_TITULAR'] = titulares_df['CPF_CNPJ_TITULAR'].str.ljust(14, ' ')
titulares_df['NOME_TITULAR'] = titulares_df['NOME_TITULAR'].str.ljust(80, ' ')
titulares_df['NOME_DOC_IDENTIFICACAO'] = titulares_df['NOME_DOC_IDENTIFICACAO'].str.ljust(50, ' ') 
titulares_df['NUMERO_DOC_IDENTIFICACAO'] = titulares_df['NUMERO_DOC_IDENTIFICACAO'].str.ljust(20, ' ')
titulares_df['ENDERECO_LOGRADOURO'] = titulares_df['ENDERECO_LOGRADOURO'].str.ljust(80, ' ')
titulares_df['ENDERECO_CIDADE'] = titulares_df['ENDERECO_CIDADE'].str.ljust(40, ' ')
titulares_df['ENDERECO_UF'] = titulares_df['ENDERECO_UF'].str.ljust(2, ' ')
titulares_df['ENDERECO_PAIS'] = titulares_df['ENDERECO_PAIS'].str.ljust(40, ' ')
titulares_df['ENDERECO_CEP'] = titulares_df['ENDERECO_CEP'].str.ljust(8, ' ')
titulares_df['TELEFONE_PESSOA'] = titulares_df['TELEFONE_PESSOA'].str.ljust(20, ' ')
titulares_df['VALOR_RENDA'] = titulares_df['VALOR_RENDA'].str.ljust(14, ' ')
titulares_df['DATA_ATUALIZACAO_RENDA'] = titulares_df['DATA_ATUALIZACAO_RENDA'].str.ljust(8, ' ') 
titulares_df['DATA_INICIO_RELACIONAMENTO_CONTA'] = titulares_df['DATA_INICIO_RELACIONAMENTO_CONTA'].str.ljust(8, ' ') 
titulares_df['DATA_FIM_RELACIONAMENTO_CONTA'] = titulares_df['DATA_FIM_RELACIONAMENTO_CONTA'].str.ljust(8, ' ')
# titulares_df['NUMERO_CONTA'].str.len().drop_duplicates() # to check len

# Generating archive names + rows
column_names = ["REPORT", "NOME_ARQUIVO", "LINHA_ARQUIVO"]
titulares_df_txt = pd.DataFrame(columns = column_names)
titulares_df_txt['REPORT'] = titulares_df['HAD_REPORT']
titulares_df_txt['NOME_ARQUIVO'] = titulares_df['OFICIO_NUMBER']
titulares_df_txt['LINHA_ARQUIVO'] = titulares_df[['NUMERO_BANCO', 'NUMERO_AGENCIA', 
                                                  'NUMERO_CONTA', 'TIPO_CONTA',
                                                  'TIPO_TITULAR', 'PESSOA_INVESTIGADA', 'TIPO_PESSOA_TITULAR',
                                                  'CPF_CNPJ_TITULAR', 'NOME_TITULAR', 'NOME_DOC_IDENTIFICACAO',
                                                  'NUMERO_DOC_IDENTIFICACAO', 'ENDERECO_LOGRADOURO', 'ENDERECO_CIDADE',
                                                  'ENDERECO_UF', 'ENDERECO_PAIS', 'ENDERECO_CEP', 'TELEFONE_PESSOA',
                                                  'VALOR_RENDA', 'DATA_ATUALIZACAO_RENDA',
                                                  'DATA_INICIO_RELACIONAMENTO_CONTA','DATA_FIM_RELACIONAMENTO_CONTA']].apply(lambda x: '\t'.join(x), axis=1)
titulares_df_txt = titulares_df_txt[titulares_df_txt['REPORT'] == "002"]
titulares_df_csv = titulares_df.merge(oficios_info_df, on='OFICIO_NUMBER', how='left')
titulares_df_csv = titulares_df_csv[titulares_df_csv['HAD_REPORT'] == "002"]
titulares_df_csv = titulares_df_csv.astype({'HAD_REPORT': str,
                                          'OFICIO_NUMBER': str,
                                          'NUMERO_BANCO': str,
                                          'NUMERO_AGENCIA': str,
                                          'NUMERO_CONTA': str,
                                          'TIPO_CONTA': str,
                                          'TIPO_TITULAR': str,
                                          'PESSOA_INVESTIGADA': str,
                                          'TIPO_PESSOA_TITULAR': str,
                                          'CPF_CNPJ_TITULAR': str,
                                          'NOME_TITULAR': str,
                                          'NOME_DOC_IDENTIFICACAO': str,
                                          'NUMERO_DOC_IDENTIFICACAO': str,
                                          'ENDERECO_LOGRADOURO': str, 
                                          'ENDERECO_CIDADE': str,
                                          'ENDERECO_UF': str,
                                          'ENDERECO_PAIS': str,
                                          'ENDERECO_CEP': str,
                                          'TELEFONE_PESSOA': str,
                                          'VALOR_RENDA': str,
                                          'DATA_ATUALIZACAO_RENDA': str,
                                          'DATA_INICIO_RELACIONAMENTO_CONTA': str,
                                          'DATA_FIM_RELACIONAMENTO_CONTA': str,
                                          'CARGO_MAGISTRADO': str,
                                          'DATA_OFICIO': str,
                                          'EMAIL_CASO': str,
                                          'NOME_MAGISTRADO': str,
                                          'NOME_TRIBUNAL': str,
                                          'NUMERO_PROCESSO': str})
print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
# Generating 'TITULARES' .txt
for row in titulares_df_txt.groupby('NOME_ARQUIVO'):
    row[1]['LINHA_ARQUIVO'].to_csv('/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/'+str(row[0]).strip('[]').strip("'")+'_TITULARES.txt', 
                                      index = False, 
                                      header = False,
                                      quotechar= "",
                                      sep = "\n",
                                      escapechar = '\\',
                                      quoting=csv.QUOTE_NONE)

# Append excel file with sheet = TITULARES to be sent @email.
titulares_grouped = titulares_df_csv.groupby('OFICIO_NUMBER')
for ti_titles, ti_rows in titulares_grouped:
    with pd.ExcelWriter('/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/'+str(ti_titles)+'.xlsx',  mode='a') as writer:  
        ti_rows.to_excel(writer, sheet_name='TITULARES', index= False, header = True)
        
print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
# Starting generation of 'EXTRATO' layout
extrato = oficios_df[['OFICIO_NUMBER', 'MOV_MOVE_ID', 'BANK_ID', 'WIT_BANK_ACC_BRANCH_ID',
                        'WIT_BANK_ACC_NUMBER', 'ACCOUNT_TYPE', 'MOV_AMOUNT_CALC', 'MOV_CREATED_DT_CALC',
                         'MOV_DETAIL', 'MOV_TYPE_ID', 'CUS_CUST_DOC_NUMBER', 'PAY_OPERATION_TYPE_ID', 
                         'HAD_REPORT']].drop_duplicates()
extrato = extrato[~extrato['CUS_CUST_DOC_NUMBER'].isna()]
extrato['OFICIO_NUMBER'] = extrato['OFICIO_NUMBER']
extrato['HAD_REPORT'] = extrato['HAD_REPORT']
extrato['CODIGO_CHAVE_EXTRATO'] = extrato['MOV_MOVE_ID'].fillna('0').astype({'MOV_MOVE_ID': int}).astype(str)
#extrato['NUMERO_BANCO'] = extrato['BANK_ID'].fillna('033')
extrato['NUMERO_BANCO'] = "323"
#extrato['NUMERO_AGENCIA'] = extrato['WIT_BANK_ACC_BRANCH_ID'].fillna('2167')
extrato['NUMERO_AGENCIA'] = "0001"
#extrato['NUMERO_CONTA'] = extrato['WIT_BANK_ACC_NUMBER'].fillna('130010391')
extrato['NUMERO_CONTA'] = "10573521"
# extrato['TIPO_CONTA'] = np.where(extrato['NUMERO_BANCO'] == "323", '4',
#                    np.where(extrato['ACCOUNT_TYPE'] == "checking_account", '1',
#                    np.where(extrato['ACCOUNT_TYPE'] == "savings_account", '2', '4')))
extrato['TIPO_CONTA'] = "4"
extrato['DATA_LANCAMENTO'] = extrato['MOV_CREATED_DT_CALC'].fillna('')
extrato['NUMERO_DOCUMENTO'] = ""
extrato['DESCRICAO_LANCAMENTO'] = np.where((extrato['MOV_DETAIL'] == "money_transfer") & (extrato['MOV_TYPE_ID'] == 'expense'), 'Envio de dinheiro entre contas Mercado Pago', 
                                  np.where((extrato['MOV_DETAIL'] == "withdraw") & (extrato['MOV_TYPE_ID'] == 'expense'), 'Envio de TED/DOC interbancaria', 
                                  np.where((extrato['MOV_DETAIL'] == "money_transfer") & (extrato['MOV_TYPE_ID'] == 'income'), 'Recebimento de dinheiro entre contas Mercado Pago', 
                                  np.where((extrato['MOV_DETAIL'] == "payment") & (extrato['MOV_TYPE_ID'] == 'income'), 'Recebimento de dinheiro entre contas Mercado Pago', 
                                  np.where((extrato['MOV_DETAIL'] == "payment") & (extrato['MOV_TYPE_ID'] == 'expense'), 'Pagamento de dinheiro entre contas Mercado Pago', 
                                  np.where((extrato['MOV_DETAIL'] == "payment") & (extrato['MOV_TYPE_ID'] == 'fund'), 'Recebimento de dinheiro entre contas Mercado Pago', 
                                  np.where((extrato['MOV_DETAIL'] == "account_fund") & (extrato['MOV_TYPE_ID'] == 'fund'), 'Recebimento de dinheiro entre contas Mercado Pago', 
                                  np.where((extrato['MOV_DETAIL'] == "cellphone_recharge") & (extrato['MOV_TYPE_ID'] == 'expense'), 'Pagamento de despesas entre contas Mercado Pago', 
                                   ''))))))))
extrato['TIPO_LANCAMENTO'] = np.where((extrato['MOV_DETAIL'] == "money_transfer") & (extrato['MOV_TYPE_ID'] == 'expense'), '117', 
                             np.where((extrato['MOV_DETAIL'] == "withdraw") & (extrato['MOV_TYPE_ID'] == 'expense'), '120', 
                             np.where((extrato['MOV_DETAIL'] == "money_transfer") & (extrato['MOV_TYPE_ID'] == 'income'), '213', 
                             np.where((extrato['MOV_DETAIL'] == "payment") & (extrato['MOV_TYPE_ID'] == 'income'), '218', 
                             np.where((extrato['MOV_DETAIL'] == "payment") & (extrato['MOV_TYPE_ID'] == 'expense'), '117', 
                             np.where((extrato['MOV_DETAIL'] == "payment") & (extrato['MOV_TYPE_ID'] == 'fund'), '217', 
                             np.where((extrato['MOV_DETAIL'] == "account_fund") & (extrato['MOV_TYPE_ID'] == 'fund'), '217', 
                             np.where((extrato['MOV_DETAIL'] == "cellphone_recharge") & (extrato['MOV_TYPE_ID'] == 'expense'), '117', 
                                ''))))))))
extrato['VALOR_LANCAMENTO'] = extrato['MOV_AMOUNT_CALC'].fillna('')
extrato['NATUREZA_LANCAMENTO'] = np.where((extrato['MOV_DETAIL'] == "money_transfer") & (extrato['MOV_TYPE_ID'] == 'expense'), 'D', 
                                 np.where((extrato['MOV_DETAIL'] == "withdraw") & (extrato['MOV_TYPE_ID'] == 'expense'), 'D', 
                                 np.where((extrato['MOV_DETAIL'] == "money_transfer") & (extrato['MOV_TYPE_ID'] == 'income'), 'C', 
                                 np.where((extrato['MOV_DETAIL'] == "payment") & (extrato['MOV_TYPE_ID'] == 'income'), 'C', 
                                 np.where((extrato['MOV_DETAIL'] == "payment") & (extrato['MOV_TYPE_ID'] == 'expense'), 'D', 
                                 np.where((extrato['MOV_DETAIL'] == "payment") & (extrato['MOV_TYPE_ID'] == 'fund'), 'C', 
                                 np.where((extrato['MOV_DETAIL'] == "account_fund") & (extrato['MOV_TYPE_ID'] == 'fund'), 'C', 
                                 np.where((extrato['MOV_DETAIL'] == "cellphone_recharge") & (extrato['MOV_TYPE_ID'] == 'expense'), 'D', 
                                 ''))))))))
extrato['VALOR_SALDO'] = ""
extrato['NATUREZA_SALDO'] = extrato['NATUREZA_LANCAMENTO'].fillna('')
extrato['LOCAL_TRANSACAO'] = "Internet"

extrato = extrato[['HAD_REPORT', 'OFICIO_NUMBER', 'CODIGO_CHAVE_EXTRATO', 'NUMERO_BANCO', 'NUMERO_AGENCIA', 
                  'NUMERO_CONTA', 'TIPO_CONTA', 'DATA_LANCAMENTO', 'NUMERO_DOCUMENTO',
                  'DESCRICAO_LANCAMENTO', 'TIPO_LANCAMENTO', 'VALOR_LANCAMENTO',
                  'NATUREZA_LANCAMENTO', 'VALOR_SALDO', 'NATUREZA_SALDO', 'LOCAL_TRANSACAO']].drop_duplicates()
extrato_df = pd.DataFrame(extrato)

# Correcting 'EXTRATO' layout
    # CODIGO_CHAVE_EXTRATO = 18 char
    # NUMERO_BANCO = 3 char
    # NUMERO_AGENCIA = 4 char
    # NUMERO_CONTA = 20 char
    # TIPO_CONTA = 1 char
    # DATA_LANCAMENTO = 8 char, DDMMYYYY
    # NUMERO_DOCUMENTO = 20 char
    # DESCRICAO_LANCAMENTO = 50 char
    # TIPO_LANCAMENTO = 3 char
    # VALOR_LANCAMENTO = 14 char
    # NATUREZA_LANCAMENTO = 1 char
    # NATUREZA_SALDO = 1 char
    # LOCAL_TRANSACAO = 80 char
    
extrato_df['CODIGO_CHAVE_EXTRATO'] = extrato_df['CODIGO_CHAVE_EXTRATO'].str.ljust(18, ' ')
extrato_df['NUMERO_BANCO'] = extrato_df['NUMERO_BANCO'].str.ljust(3, ' ')
extrato_df['NUMERO_AGENCIA'] = extrato_df['NUMERO_AGENCIA'].str.ljust(4, ' ')
extrato_df['NUMERO_CONTA'] = extrato_df['NUMERO_CONTA'].str.ljust(20, ' ')
extrato_df['TIPO_CONTA'] = extrato_df['TIPO_CONTA'].str.ljust(1, ' ')
extrato_df['DATA_LANCAMENTO'] = extrato_df['DATA_LANCAMENTO'].str.ljust(8, ' ')
extrato_df['NUMERO_DOCUMENTO'] = extrato_df['NUMERO_DOCUMENTO'].str.ljust(20, '0')
extrato_df['DESCRICAO_LANCAMENTO'] = extrato_df['DESCRICAO_LANCAMENTO'].str.ljust(50, ' ')
extrato_df['TIPO_LANCAMENTO'] = extrato_df['TIPO_LANCAMENTO'].str.ljust(3, ' ')
extrato_df['VALOR_LANCAMENTO'] = extrato_df['VALOR_LANCAMENTO'].str.ljust(14, ' ')
extrato_df['NATUREZA_LANCAMENTO'] = extrato_df['NATUREZA_LANCAMENTO'].str.ljust(1, ' ')
extrato_df['VALOR_SALDO'] = extrato_df['VALOR_SALDO'].str.ljust(14, '0')
extrato_df['NATUREZA_SALDO'] = extrato_df['NATUREZA_SALDO'].str.ljust(1, ' ')
extrato_df['LOCAL_TRANSACAO'] = extrato_df['LOCAL_TRANSACAO'].str.ljust(1, ' ')
# extrato_df['NUMERO_CONTA'].str.len().drop_duplicates() # to check len


# Generating archive names + rows
column_names = ["REPORT", "NOME_ARQUIVO", "LINHA_ARQUIVO"]
extrato_df_txt = pd.DataFrame(columns = column_names)
extrato_df_txt['REPORT'] = extrato_df['HAD_REPORT']
extrato_df_txt['NOME_ARQUIVO'] = extrato_df['OFICIO_NUMBER']
extrato_df_txt['LINHA_ARQUIVO'] = extrato_df[['CODIGO_CHAVE_EXTRATO', 'NUMERO_BANCO', 'NUMERO_AGENCIA', 
                                      'NUMERO_CONTA', 'TIPO_CONTA', 'DATA_LANCAMENTO', 'NUMERO_DOCUMENTO',
                                       'DESCRICAO_LANCAMENTO', 'TIPO_LANCAMENTO', 'VALOR_LANCAMENTO',
                                      'NATUREZA_LANCAMENTO', 'VALOR_SALDO', 'NATUREZA_SALDO', 'LOCAL_TRANSACAO']].apply(lambda x: '\t'.join(x), axis=1)
extrato_df_txt = extrato_df_txt[extrato_df_txt['REPORT'] == "002"]
extrato_df_csv = extrato_df.merge(oficios_info_df, on='OFICIO_NUMBER', how='left')
extrato_df_csv = extrato_df_csv[extrato_df_csv['HAD_REPORT'] == "002"]
extrato_df_csv = extrato_df_csv.astype({'HAD_REPORT': str,
                                          'OFICIO_NUMBER': str,
                                          'CODIGO_CHAVE_EXTRATO': str,
                                          'NUMERO_BANCO': str,
                                          'NUMERO_AGENCIA': str, 
                                          'NUMERO_CONTA': str,
                                          'TIPO_CONTA': str,
                                          'DATA_LANCAMENTO': str,
                                          'NUMERO_DOCUMENTO': str, 
                                          'DESCRICAO_LANCAMENTO': str, 
                                          'TIPO_LANCAMENTO': str,
                                          'VALOR_LANCAMENTO': str,
                                          'NATUREZA_LANCAMENTO': str, 
                                          'VALOR_SALDO': str,
                                          'NATUREZA_SALDO': str,
                                          'LOCAL_TRANSACAO': str,
                                          'CARGO_MAGISTRADO': str,
                                          'DATA_OFICIO': str, 
                                          'EMAIL_CASO': str,
                                          'NOME_MAGISTRADO': str,
                                          'NOME_TRIBUNAL': str, 
                                          'NUMERO_PROCESSO': str})
print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
# Generating 'EXTRATO' .txt
#extrato_df_txt_002 = extrato_df_txt[extrato_df_txt['REPORT'] == "002"]
for row in extrato_df_txt.groupby('NOME_ARQUIVO'):
    row[1]['LINHA_ARQUIVO'].to_csv('/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/'+str(row[0]).strip('[]').strip("'")+'_EXTRATO.txt', 
                                      index = False, 
                                      header = False,
                                      quotechar= "",
                                      sep = "\n",
                                      escapechar = '\\',
                                      quoting=csv.QUOTE_NONE)
    
# Append excel file with sheet = EXTRATO to be sent @email.
extrato_grouped = extrato_df_csv.groupby('OFICIO_NUMBER')
for ex_titles, ex_rows in extrato_grouped:
    with pd.ExcelWriter('/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/'+str(ex_titles)+'.xlsx',  mode='a') as writer:  
        ex_rows.to_excel(writer, sheet_name='EXTRATO', index= False, header = True)
        
print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
# Starting generation of 'ORIGEM DESTINO' layout
od = oficios_df[['OFICIO_NUMBER', 'MOV_MOVE_ID', 'BANK_ID', 'WIT_BANK_ACC_BRANCH_ID', 'PAY_REASON_ID',
                        'WIT_BANK_ACC_NUMBER', 'ACCOUNT_TYPE', 'MOV_AMOUNT_CALC', 'CUS_CUST_DOC_NUMBER_SEL', 'MOV_CREATED_DT_CALC',
                         'MOV_DETAIL', 'MOV_TYPE_ID', 'PAY_OPERATION_TYPE_ID', 'CUS_FULL_NAME_SEL', 'CUS_CUST_DOC_TYPE_SEL',
                         'HAD_REPORT']].drop_duplicates()
#od = od[~od['CUS_CUST_DOC_NUMBER'].isna()]
od['OFICIO_NUMBER'] = od['OFICIO_NUMBER']
od['HAD_REPORT'] = od['HAD_REPORT']
od['CODIGO_CHAVE_OD'] = np.arange(len(od))+1
od['CODIGO_CHAVE_OD'] = od['CODIGO_CHAVE_OD'].astype(str)
od['CODIGO_CHAVE_EXTRATO'] = od['MOV_MOVE_ID'].fillna('0').astype({'MOV_MOVE_ID': int}).astype(str)
od['VALOR_TRANSACAO'] = od['MOV_AMOUNT_CALC'].fillna('')
od['NUMERO_DOCUMENTO_TRANSACAO'] = ""
od['NUMERO_BANCO_OD'] = od['BANK_ID'].fillna('323')
od['NUMERO_AGENCIA_OD'] = od['WIT_BANK_ACC_BRANCH_ID'].fillna('0001')
od['NUMERO_CONTA_OD'] = od['WIT_BANK_ACC_NUMBER'].fillna('10573521')
od['TIPO_CONTA_OD'] = np.where(od['NUMERO_BANCO_OD'] == "323", '4',
                    np.where(od['ACCOUNT_TYPE'] == "checking_account", '1',
                    np.where(od['ACCOUNT_TYPE'] == "savings_account", '2', '4')))
#od['TIPO_PESSOA_OD'] = np.where(od['CUS_CUST_DOC_TYPE_SEL'] == "CPF", "1", "2")
od['TIPO_PESSOA_OD'] = np.where(od['CUS_CUST_DOC_TYPE_SEL'].isna(), 
                        np.where(od['CUS_CUST_DOC_NUMBER_SEL'].fillna('').apply(len) == 11, '1', '2')
                                 ,np.where(od['CUS_CUST_DOC_TYPE_SEL'] == "CPF", "1", "2"))
od['CPF_CNPJ_OD'] = od['CUS_CUST_DOC_NUMBER_SEL'].fillna('')
od['NOME_PESSOA_OD'] = od['CUS_FULL_NAME_SEL'].fillna('')
od['NOME_DOC_IDENTIFICACAO_OD'] = ""
od['NUMERO_DOC_IDENTIFICACAO_OD'] = ""
od['CODIGO_DE_BARRAS'] = ""
od['NOME_ENDOSSANTE_CHEQUE'] = ""
od['DOC_ENDOSSANTE_CHEQUE'] = ""
od['SITUACAO_IDENTIFICACAO'] = "0"
od['OBSERVACAO'] = od['PAY_REASON_ID'].fillna('')



od = od[['HAD_REPORT', 'OFICIO_NUMBER', 'CODIGO_CHAVE_OD', 'CODIGO_CHAVE_EXTRATO',
        'VALOR_TRANSACAO', 'NUMERO_DOCUMENTO_TRANSACAO', 'NUMERO_BANCO_OD',
        'NUMERO_AGENCIA_OD', 'NUMERO_CONTA_OD', 'TIPO_CONTA_OD',
        'TIPO_PESSOA_OD', 'CPF_CNPJ_OD', 'NOME_PESSOA_OD',
        'NOME_DOC_IDENTIFICACAO_OD', 'NUMERO_DOC_IDENTIFICACAO_OD', 'CODIGO_DE_BARRAS', 'NOME_ENDOSSANTE_CHEQUE',
        'DOC_ENDOSSANTE_CHEQUE', 'SITUACAO_IDENTIFICACAO', 'OBSERVACAO']].drop_duplicates()
od_df = pd.DataFrame(od)

# Correcting 'ORIGEM DESTINO' layout
    # CODIGO_CHAVE_OD = 18 char
    # CODIGO_CHAVE_EXTRATO = 18 char
    # VALOR_TRANSACAO = 14 char
    # NUMERO_DOCUMENTO_TRANSACAO = 20 char
    # NUMERO_BANCO_OD = 3 char
    # NUMERO_AGENCIA_OD = 4 char
    # NUMERO_CONTA_OD = 20 char
    # TIPO_CONTA_OD = 1 char
    # TIPO_PESSOA_OD = 1 char
    # CPF_CNPJ_OD = 14 char
    # NOME_PESSOA_OD = 80 char
    # NOME_DOC_IDENTIFICACAO_OD = 20 char
    # NUMERO_DOC_IDENTIFICACAO_OD = 20 char
    # CODIGO_DE_BARRAS = 100 char
    # NOME_ENDOSSANTE_CHEQUE = 80 char
    # DOC_ENDOSSANTE_CHEQUE = 50 char
    # SITUACAO_IDENTIFICACAO = 1 char
    # OBSERVACAO = 250 char
    
od_df['CODIGO_CHAVE_OD'] = od_df['CODIGO_CHAVE_OD'].str.ljust(18, ' ')
od_df['CODIGO_CHAVE_EXTRATO'] = od_df['CODIGO_CHAVE_EXTRATO'].str.ljust(18, ' ')
od_df['VALOR_TRANSACAO'] = od_df['VALOR_TRANSACAO'].str.ljust(14, ' ')
od_df['NUMERO_DOCUMENTO_TRANSACAO'] = od_df['NUMERO_DOCUMENTO_TRANSACAO'].str.ljust(20, ' ')
od_df['NUMERO_BANCO_OD'] = od_df['NUMERO_BANCO_OD'].str.ljust(3, ' ')
od_df['NUMERO_AGENCIA_OD'] = od_df['NUMERO_AGENCIA_OD'].str.ljust(4, ' ')
od_df['NUMERO_CONTA_OD'] = od_df['NUMERO_CONTA_OD'].str.ljust(20, ' ')
od_df['TIPO_CONTA_OD'] = od_df['TIPO_CONTA_OD'].str.ljust(1, ' ')
od_df['TIPO_PESSOA_OD'] = od_df['TIPO_PESSOA_OD'].str.ljust(1, ' ')
od_df['CPF_CNPJ_OD'] = od_df['CPF_CNPJ_OD'].str.ljust(14, ' ')
od_df['NOME_PESSOA_OD'] = od_df['NOME_PESSOA_OD'].str.ljust(14, ' ')
od_df['NOME_DOC_IDENTIFICACAO_OD'] = od_df['NOME_DOC_IDENTIFICACAO_OD'].str.ljust(20, ' ')
od_df['NUMERO_DOC_IDENTIFICACAO_OD'] = od_df['NUMERO_DOC_IDENTIFICACAO_OD'].str.ljust(20, ' ')
od_df['CODIGO_DE_BARRAS'] = od_df['CODIGO_DE_BARRAS'].str.ljust(100, ' ')
od_df['NOME_ENDOSSANTE_CHEQUE'] = od_df['NOME_ENDOSSANTE_CHEQUE'].str.ljust(80, ' ')
od_df['DOC_ENDOSSANTE_CHEQUE'] = od_df['DOC_ENDOSSANTE_CHEQUE'].str.ljust(50, ' ')
od_df['SITUACAO_IDENTIFICACAO'] = od_df['SITUACAO_IDENTIFICACAO'].str.ljust(1, ' ')
od_df['OBSERVACAO'] = od['OBSERVACAO'].str.ljust(250, ' ')
# od_df['OBSERVACAO'].str.len().drop_duplicates() # to check len



# Generating archive names + rows
column_names = ["REPORT", "NOME_ARQUIVO", "LINHA_ARQUIVO"]
od_df_txt = pd.DataFrame(columns = column_names)
od_df_txt['REPORT'] = od_df['HAD_REPORT']
od_df_txt['NOME_ARQUIVO'] = od_df['OFICIO_NUMBER']
od_df_txt['LINHA_ARQUIVO'] = od_df[['CODIGO_CHAVE_OD', 'CODIGO_CHAVE_EXTRATO',
        'VALOR_TRANSACAO', 'NUMERO_DOCUMENTO_TRANSACAO', 'NUMERO_BANCO_OD',
        'NUMERO_AGENCIA_OD', 'NUMERO_CONTA_OD', 'TIPO_CONTA_OD',
        'TIPO_PESSOA_OD', 'CPF_CNPJ_OD', 'NOME_PESSOA_OD',
        'NOME_DOC_IDENTIFICACAO_OD', 'NUMERO_DOC_IDENTIFICACAO_OD', 'CODIGO_DE_BARRAS', 'NOME_ENDOSSANTE_CHEQUE',
        'DOC_ENDOSSANTE_CHEQUE', 'SITUACAO_IDENTIFICACAO', 'OBSERVACAO']].apply(lambda x: '\t'.join(x), axis=1)
od_df_txt = od_df_txt[od_df_txt['REPORT'] == "002"]
od_df_csv = od_df.merge(oficios_info_df, on='OFICIO_NUMBER', how='left')
od_df_csv = od_df_csv[od_df_csv['HAD_REPORT'] == "002"]
od_df_csv = od_df_csv.astype({'HAD_REPORT': str,
                              'OFICIO_NUMBER': str,
                              'CODIGO_CHAVE_OD': str,
                              'CODIGO_CHAVE_EXTRATO': str,
                              'VALOR_TRANSACAO': str,
                              'NUMERO_DOCUMENTO_TRANSACAO': str,
                              'NUMERO_BANCO_OD': str,
                              'NUMERO_AGENCIA_OD': str,
                              'NUMERO_CONTA_OD': str,
                              'TIPO_CONTA_OD': str,
                              'TIPO_PESSOA_OD': str,
                              'CPF_CNPJ_OD': str,
                              'NOME_PESSOA_OD': str,
                              'NOME_DOC_IDENTIFICACAO_OD': str,
                              'NUMERO_DOC_IDENTIFICACAO_OD': str,
                              'CODIGO_DE_BARRAS': str,
                              'NOME_ENDOSSANTE_CHEQUE': str,
                              'DOC_ENDOSSANTE_CHEQUE': str,
                              'SITUACAO_IDENTIFICACAO': str,
                              'OBSERVACAO': str,
                              'CARGO_MAGISTRADO': str,
                              'DATA_OFICIO': str,
                              'EMAIL_CASO': str,
                              'NOME_MAGISTRADO': str,
                              'NOME_TRIBUNAL': str,
                              'NUMERO_PROCESSO': str})
print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
# Generating 'ORIGEM DESTINO' .txt
#od_df_txt_002 = od_df_txt[od_df_txt['REPORT'] == "002"]
for row in od_df_txt.groupby('NOME_ARQUIVO'):
    row[1]['LINHA_ARQUIVO'].to_csv('/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/'+str(row[0]).strip('[]').strip("'")+'_ORIGEM_DESTINO.txt', 
                                      index = False, 
                                      header = False,
                                      quotechar= "",
                                      sep = "\n",
                                      escapechar = '\\',
                                      quoting=csv.QUOTE_NONE)

# Append excel file with sheet = ORIGEM_DESTINO to be sent @email.
od_grouped = od_df_csv.groupby('OFICIO_NUMBER')
for od_titles, od_rows in od_grouped:
    with pd.ExcelWriter('/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/'+str(od_titles)+'.xlsx',  mode='a') as writer:  
        od_rows.to_excel(writer, sheet_name='ORIGEM_DESTINO', index= False, header = True)
        
print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
# Starting generation of 'INVESTIGADOS' layout
inv = oficios_df[['OFICIO_NUMBER', 'MOV_MOVE_ID', 'BANK_ID', 'WIT_BANK_ACC_BRANCH_ID',
                        'WIT_BANK_ACC_NUMBER', 'ACCOUNT_TYPE', 'MOV_AMOUNT_CALC', 'MOV_CREATED_DT_CALC',
                         'MOV_DETAIL', 'MOV_TYPE_ID', 'PAY_OPERATION_TYPE_ID', 
                         'CUS_CUST_DOC_NUMBER', 'CUS_CUST_DOC_TYPE', 'CUS_FULL_NAME',
                         'HAD_REPORT']].drop_duplicates()
#inv = inv[~inv['CUS_CUST_DOC_NUMBER_BUY'].isna()]

cus_full_name = inv[['CUS_FULL_NAME']].groupby(inv['CUS_CUST_DOC_NUMBER']).nth(0).reset_index()
cus_full_name['CUS_FULL_NAME_CORRECTED'] = cus_full_name['CUS_FULL_NAME']
del cus_full_name['CUS_FULL_NAME']
inv = inv.merge(doc_info_df, on=['OFICIO_NUMBER', 'CUS_CUST_DOC_NUMBER'], how='left')
inv = inv.merge(cus_full_name, on=['CUS_CUST_DOC_NUMBER'], how = 'left')
inv['OFICIO_NUMBER'] = inv['OFICIO_NUMBER']
inv['HAD_REPORT'] = inv['HAD_REPORT']
inv['TIPO_PESSOA_OD'] = np.where(inv['CUS_CUST_DOC_TYPE'].isna(), 
                        np.where(inv['CUS_CUST_DOC_NUMBER'].apply(len) == 11, '1', '2')
                                 ,np.where(inv['CUS_CUST_DOC_TYPE'] == "CPF", "1", "2"))
inv['CPF_CNPJ_OD'] = inv['CUS_CUST_DOC_NUMBER'].fillna('Nome não encontrado')
inv['NOME_PESSOA_OD'] = inv['CUS_FULL_NAME_CORRECTED'].fillna('Nome não encontrado')
inv['POSSUI_RELACIONAMENTO'] = np.where(inv['HAD_REPORT'] == "002", "1", "0")
inv['POSSUI_CONTA'] = np.where(inv['HAD_REPORT'] == "002", "1", "0")
inv['POSSUI_BDV'] = np.where(inv['HAD_REPORT'] == "002", "1", "0")
inv['OBSERVACAO'] = ""
inv['DATA_INICIO_AFASTAMENTO'] = inv['RANGE_MIN_CALC']
inv['DATA_FIM_AFASTAMENTO'] = inv['RANGE_MAX_CALC']


inv = inv[['HAD_REPORT', 'OFICIO_NUMBER', 'TIPO_PESSOA_OD', 
          'CPF_CNPJ_OD', 'NOME_PESSOA_OD', 'POSSUI_RELACIONAMENTO', 'POSSUI_CONTA', 'POSSUI_BDV',
          'OBSERVACAO', 'DATA_INICIO_AFASTAMENTO', 'DATA_FIM_AFASTAMENTO']].drop_duplicates()
inv_df = pd.DataFrame(inv)

# Correcting 'INVESTIGADOS' layout
    # TIPO_PESSOA_OD = 1 char
    # CPF_CNPJ_OD = 14 char
    # NOME_PESSOA_OD = 80 char
    # POSSUI_RELACIONAMENTO = 1 char
    # POSSUI_CONTA = 1 char
    # POSSUI_BDV = 1 char
    # OBSERVACAO = 250 char
    # DATA_INICIO_AFASTAMENTO = 8 char, DDMMYYYY
    # DATA_FIM_AFASTAMENTO = 8 char, DDMMYYYY
    

    
inv_df['TIPO_PESSOA_OD'] = inv_df['TIPO_PESSOA_OD'].str.ljust(1, ' ')
inv_df['CPF_CNPJ_OD'] = inv_df['CPF_CNPJ_OD'].str.ljust(14, ' ')
inv_df['NOME_PESSOA_OD'] = inv_df['NOME_PESSOA_OD'].str.ljust(80, ' ')
inv_df['POSSUI_RELACIONAMENTO'] = inv_df['POSSUI_RELACIONAMENTO'].str.ljust(1, ' ')
inv_df['POSSUI_CONTA'] = inv_df['POSSUI_CONTA'].str.ljust(1, ' ')
inv_df['POSSUI_BDV'] = inv_df['POSSUI_BDV'].str.ljust(1, ' ')
inv_df['OBSERVACAO'] = inv_df['OBSERVACAO'].str.ljust(250, ' ')
inv_df['DATA_INICIO_AFASTAMENTO'] = inv_df['DATA_INICIO_AFASTAMENTO'].str.ljust(8, ' ')
inv_df['DATA_FIM_AFASTAMENTO'] = inv_df['DATA_FIM_AFASTAMENTO'].str.ljust(8, ' ')
# inv_df['OBSERVACAO'].str.len().drop_duplicates() # to check len


# Generating archive names + rows
column_names = ["REPORT", "NOME_ARQUIVO", "LINHA_ARQUIVO"]
inv_df_txt = pd.DataFrame(columns = column_names)
inv_df_txt['REPORT'] = inv_df['HAD_REPORT']
inv_df_txt['NOME_ARQUIVO'] = inv_df['OFICIO_NUMBER']
inv_df_txt['LINHA_ARQUIVO'] = inv_df[['TIPO_PESSOA_OD', 'CPF_CNPJ_OD', 'NOME_PESSOA_OD',
                                      'POSSUI_RELACIONAMENTO', 'POSSUI_CONTA', 'POSSUI_BDV', 'OBSERVACAO',
                                      'DATA_INICIO_AFASTAMENTO', 'DATA_FIM_AFASTAMENTO']].apply(lambda x: '\t'.join(x), axis=1)
inv_df_txt = inv_df_txt[inv_df_txt['REPORT'] == "002"]
inv_df_csv = inv_df.merge(oficios_info_df, on='OFICIO_NUMBER', how='left')
inv_df_csv = inv_df_csv[inv_df_csv['HAD_REPORT'] == "002"]
inv_df_csv = inv_df_csv.astype({'HAD_REPORT': str,
                              'OFICIO_NUMBER': str,
                              'TIPO_PESSOA_OD': str,
                              'CPF_CNPJ_OD': str,
                              'NOME_PESSOA_OD': str,
                              'POSSUI_RELACIONAMENTO': str,
                              'POSSUI_CONTA': str,
                              'POSSUI_BDV': str,
                              'OBSERVACAO': str,
                              'DATA_INICIO_AFASTAMENTO': str,
                              'DATA_FIM_AFASTAMENTO': str})
print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
# Generating 'INVESTIGADOS' .txt
for row in inv_df_txt.groupby('NOME_ARQUIVO'):
    row[1]['LINHA_ARQUIVO'].to_csv('/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/'+str(row[0]).strip('[]').strip("'")+'_INVESTIGADO.txt', 
                                      index = False, 
                                      header = False,
                                      quotechar= "",
                                      sep = "\n",
                                      escapechar = '\\',
                                      quoting=csv.QUOTE_NONE)

# Append excel file with sheet = INVESTIGADO to be sent @email.
inv_grouped = inv_df_csv.groupby('OFICIO_NUMBER')
for inv_titles, inv_rows in inv_grouped:
    with pd.ExcelWriter('/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/'+str(inv_titles)+'.xlsx',  mode='a') as writer:  
        inv_rows.to_excel(writer, sheet_name='INVESTIGADO', index= False, header = True)
        
print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
#of = ["059-PCRO-000028-45"]
#oficios_email = oficios_info_df[~oficios_info_df.OFICIO_NUMBER.isin(of)]
#oficios_email = oficios_info_df[oficios_info_df.OFICIO_NUMBER.isin(of)]
oficios_email = oficios_info_df
# oficios_email['EMAIL_CASO_sCALC'] = np.where(oficios_email['EMAIL_CASO'] == '04vara.pa@trfl.jus.br',
#                                              'raphael.dayan@mercadopago.com.br',
#                                              'dayanrapha@gmail.com')
#oficios_email['EMAIL_CASO_CALC'] = 'daniel.damasceno@mercadolivre.com'
#oficios_email['EMAIL_CASO_CALC'] = 'ext_gbaranda@mercadolivre.com'
oficios_email['EMAIL_CASO_CALC'] = 'ext_svieirad@mercadolibre.com'
#oficios_email['EMAIL_CASO_CALC'] = 'gabriel.matias@mercadolivre.com'
#oficios_email['EMAIL_CASO_CALC'] = 'andrielly.ribeiro@mercadolivre.com'
#oficios_email['EMAIL_CASO_CALC'] = 'raphael.dayan@mercadopago.com.br'
oficios_email['ARCHIVE_NAME_XLSX'] = oficios_email['OFICIO_NUMBER'].map(str) + '.xlsx'
oficios_email['LINK_ATTACH_XLSX'] = '/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/' + oficios_email['OFICIO_NUMBER'].map(str) + '.xlsx'
oficios_email['ARCHIVE_NAME_AG'] = oficios_email['OFICIO_NUMBER'].map(str) + '_AGENCIAS.txt'
oficios_email['LINK_ATTACH_AG'] = '/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/' + oficios_email['OFICIO_NUMBER'].map(str) + '_AGENCIAS.txt'
oficios_email['ARCHIVE_NAME_CO'] = oficios_email['OFICIO_NUMBER'].map(str) + '_CONTAS.txt'
oficios_email['LINK_ATTACH_CO'] = '/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/' + oficios_email['OFICIO_NUMBER'].map(str) + '_CONTAS.txt'
oficios_email['ARCHIVE_NAME_EX'] = oficios_email['OFICIO_NUMBER'].map(str) + '_EXTRATO.txt'
oficios_email['LINK_ATTACH_EX'] = '/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/' + oficios_email['OFICIO_NUMBER'].map(str) + '_EXTRATO.txt'
oficios_email['ARCHIVE_NAME_OD'] = oficios_email['OFICIO_NUMBER'].map(str) + '_ORIGEM_DESTINO.txt'
oficios_email['LINK_ATTACH_OD'] = '/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/' + oficios_email['OFICIO_NUMBER'].map(str) + '_ORIGEM_DESTINO.txt'
oficios_email['ARCHIVE_NAME_TI'] = oficios_email['OFICIO_NUMBER'].map(str) + '_TITULARES.txt'
oficios_email['LINK_ATTACH_TI'] = '/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/' + oficios_email['OFICIO_NUMBER'].map(str) + '_TITULARES.txt'
oficios_email['ARCHIVE_NAME_INV'] = oficios_email['OFICIO_NUMBER'].map(str) + '_INVESTIGADO.txt'
oficios_email['LINK_ATTACH_INV'] = '/alloc/data/fury_oficios-quebra-sigilo/notebooks/Archives/' + oficios_email['OFICIO_NUMBER'].map(str) + '_INVESTIGADO.txt'
print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
# For 2 factor auth, 
    # 1. Log-in into Gmail with your account
    # 2. Navigate to https://security.google.com/settings/security/apppasswords
    # 3. In 'select app' choose 'custom', give it an arbitrary name and press generate
    # 4. It will give you 16 chars token.
# Made based on https://realpython.com/python-send-email/


# Generate email 
import smtplib, ssl
from email.mime.text import MIMEText
from email.utils import formataddr
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

# User configuration
sender_email = 'raphael.dayan@mercadopago.com.br'
sender_name = 'Mercado Pago Simba'
password = 'jjgzaqrjkpbfjqrw'

receiver_emails = oficios_email['EMAIL_CASO_CALC']
receiver_names = oficios_email['NOME_MAGISTRADO']
receiver_cargos = oficios_email['CARGO_MAGISTRADO']
receiver_datas = oficios_email['DATA_OFICIO']
receiver_tribunais = oficios_email['NOME_TRIBUNAL']
receiver_subjects = oficios_email['OFICIO_NUMBER']
receiver_processos = oficios_email['NUMERO_PROCESSO']
receiver_attachments_xlsx = oficios_email['LINK_ATTACH_XLSX']
receiver_archives_xlsx = oficios_email['ARCHIVE_NAME_XLSX']
receiver_attachments_ag = oficios_email['LINK_ATTACH_AG']
receiver_archives_ag = oficios_email['ARCHIVE_NAME_AG']
receiver_attachments_co = oficios_email['LINK_ATTACH_CO']
receiver_archives_co = oficios_email['ARCHIVE_NAME_CO']
receiver_attachments_ex = oficios_email['LINK_ATTACH_EX']
receiver_archives_ex= oficios_email['ARCHIVE_NAME_EX']
receiver_attachments_od = oficios_email['LINK_ATTACH_OD']
receiver_archives_od= oficios_email['ARCHIVE_NAME_OD']
receiver_attachments_ti = oficios_email['LINK_ATTACH_TI']
receiver_archives_ti= oficios_email['ARCHIVE_NAME_TI']
receiver_attachments_inv = oficios_email['LINK_ATTACH_INV']
receiver_archives_inv= oficios_email['ARCHIVE_NAME_INV']

print('Sending email...')

for receiver_email, receiver_name, receiver_subject, receiver_attachment_xlsx, receiver_archive_xlsx, receiver_cargo, receiver_data, receiver_tribunal, receiver_processo, receiver_attachment_ag, receiver_archive_ag, receiver_attachment_co, receiver_archive_co, receiver_attachment_ex, receiver_archive_ex, receiver_attachment_od, receiver_archive_od, receiver_attachment_ti, receiver_archive_ti, receiver_attachment_inv, receiver_archive_inv, in zip(receiver_emails, receiver_names, receiver_subjects, receiver_attachments_xlsx, receiver_archives_xlsx, receiver_cargos, receiver_datas, receiver_tribunais, receiver_processos, receiver_attachments_ag, receiver_archives_ag, receiver_attachments_co, receiver_archives_co, receiver_attachments_ex, receiver_archives_ex, receiver_attachments_od, receiver_archives_od, receiver_attachments_ti, receiver_archives_ti, receiver_attachments_inv, receiver_archives_inv):
    # Email body
    email_body = """\
    <html>
          <body>
                <p>À {receiver_tribunal},  {receiver_cargo} {receiver_name}. <br><br>
                   Segue resposta ao Ofício Número {receiver_subject},  processo {receiver_processo} do dia  {receiver_data}.<br><br>
                    Pedimos-lhe desculpas pelo inconveniente e atraso na resposta ao Ofício supracitado.<br><br>
                    A Instituição recentemente integrou-se ao processo de CCS e BACENJUD junto ao Banco Central do Brasil, e algumas demandas ainda estão internamente em processo de estruturação e adequação para melhor atender-lhes.<br><br>
                    Neste contexto, objetivando atendê-los o mais breve possível, enviamo-lhes em anexo as informações solicitadas.<br><br>
                    Equipe Mercado Pago Simba.
               </p>
          </body>
        </html>
    """.format(receiver_name=receiver_name, receiver_subject=receiver_subject, receiver_tribunal=receiver_tribunal, receiver_cargo=receiver_cargo, receiver_data=receiver_data, receiver_processo=receiver_processo)
    # Configurating the email information
    msg = MIMEMultipart()
    msg['To'] = formataddr((receiver_name, receiver_email))
    msg['From'] = formataddr((sender_name, sender_email))
    msg['Subject'] = 'Resposta a Cooperação Técnica ' + receiver_subject
    
    msg.attach(MIMEText(email_body, 'html'))
    
    filename_xlsx = receiver_attachment_xlsx
    filename_ag = receiver_attachment_ag
    filename_co = receiver_attachment_co
    filename_ex = receiver_attachment_ex
    filename_od = receiver_attachment_od
    filename_ti = receiver_attachment_ti
    filename_inv = receiver_attachment_inv
    try:
        with open(filename_xlsx, 'rb') as attachment_xlsx:
            part_xlsx = MIMEBase("application", "octet-stream")
            part_xlsx.set_payload(attachment_xlsx.read())

        # Encode file in ASCII characters to send by email    
        encoders.encode_base64(part_xlsx)

        # Add header as key/value pair to attachment part
        part_xlsx.add_header("Content-Disposition",
                            f"attachment; filename= {receiver_archive_xlsx}",)
            
        msg.attach(part_xlsx)
        
        with open(filename_ag, 'rb') as attachment_ag:
            part_ag = MIMEBase("application", "octet-stream")
            part_ag.set_payload(attachment_ag.read())

        # Encode file in ASCII characters to send by email    
        encoders.encode_base64(part_ag)

        # Add header as key/value pair to attachment part
        part_ag.add_header("Content-Disposition",
                            f"attachment; filename= {receiver_archive_ag}",)
            
        msg.attach(part_ag)        

        with open(filename_co, 'rb') as attachment_co:
            part_co = MIMEBase("application", "octet-stream")
            part_co.set_payload(attachment_co.read())

        # Encode file in ASCII characters to send by email    
        encoders.encode_base64(part_co)

        # Add header as key/value pair to attachment part
        part_co.add_header("Content-Disposition",
                            f"attachment; filename= {receiver_archive_co}",)
            
        msg.attach(part_co)           
        
        with open(filename_ex, 'rb') as attachment_ex:
            part_ex = MIMEBase("application", "octet-stream")
            part_ex.set_payload(attachment_ex.read())

        # Encode file in ASCII characters to send by email    
        encoders.encode_base64(part_ex)

        # Add header as key/value pair to attachment part
        part_ex.add_header("Content-Disposition",
                            f"attachment; filename= {receiver_archive_ex}",)
            
        msg.attach(part_ex)           
        
        with open(filename_od, 'rb') as attachment_od:
            part_od = MIMEBase("application", "octet-stream")
            part_od.set_payload(attachment_od.read())

        # Encode file in ASCII characters to send by email    
        encoders.encode_base64(part_od)

        # Add header as key/value pair to attachment part
        part_od.add_header("Content-Disposition",
                            f"attachment; filename= {receiver_archive_od}",)
            
        msg.attach(part_od)           
        
        with open(filename_ti, 'rb') as attachment_ti:
            part_ti = MIMEBase("application", "octet-stream")
            part_ti.set_payload(attachment_ti.read())

        # Encode file in ASCII characters to send by email    
        encoders.encode_base64(part_ti)

        # Add header as key/value pair to attachment part
        part_ti.add_header("Content-Disposition",
                            f"attachment; filename= {receiver_archive_ti}",)
            
        msg.attach(part_ti)   
        
        with open(filename_inv, 'rb') as attachment_inv:
            part_inv = MIMEBase("application", "octet-stream")
            part_inv.set_payload(attachment_inv.read())

        # Encode file in ASCII characters to send by email    
        encoders.encode_base64(part_inv)

        # Add header as key/value pair to attachment part
        part_inv.add_header("Content-Disposition",
                            f"attachment; filename= {receiver_archive_inv}",)
            
        msg.attach(part_inv)  
        
    except Exception as e:
        print(f'Oops something went wrong in attachment\n{e}')

    try:
        # Creating a SMTP connection
        server = smtplib.SMTP("smtp.gmail.com", 587)
        # Encryps the email
        context = ssl.create_default_context()
        server.starttls(context=context)
        # Log in into email account
        server.login(sender_email, password)
        # Send the email
        server.sendmail(sender_email, receiver_email, msg.as_string())
        print("Email successfully sent to", receiver_email, ', Ofício:', receiver_subject)
        
        del msg
        
    except Exception as e:
        print(f'Oops something went wrong\n{e}')
    finally:
        print('Closing the server...')
        server.quit()
        
print('Job has finished sucessfully', dt.datetime.now())

In [ ]:
# Lister information 
with open("./Queries/lister.sql", "r") as f:
    lister_response = f.read().split(";")[:-1]
    
    for query in lister_response:
        lister_tbl = tera.execute_response(query)
        lister_df = pd.DataFrame(lister_tbl)
        
lister_csv = lister_df.astype({'MOV_MOVE_ID': str,
                                'PAY_PAYMENT_ID': str,
                                'PAY_REASON_ID': str,
                                'MOV_CREATED_DT': str,
                                'MOV_CREATED_DT_CALC': str,
                                'MOV_DETAIL': str,
                                'OFICIO_NUMBER': str,
                                'OFICIO_CREATED_AT': str,
                                'MOV_TYPE_ID': str,
                                'PAY_OPERATION_TYPE_ID': str,
                                'MOV_FINANCIAL_ENTITY_ID': str,
                                'WIT_WITHDRAW_ID': str,
                                'MOV_AMOUNT': str,
                                'MOV_AMOUNT_CALC': str,
                                'ACCOUNT_TYPE': str,
                                'WIT_BANK_ACC_NUMBER': str,
                                'WITHDRAWAL_STATUS': str,
                                'RECEIVING_ACCOUNT_HOLDER_NAME': str,
                                'RECEIVING_ACCOUNT_HOLDER_DOC_NUMBER': str,
                                'BANK_ID': str,
                                'WIT_BANK_ACC_BRANCH_ID': str,
                                'RECEIVING_BANK_NAME': str,
                                'PAYER_BANK_NAME': str,
                                'WITHDRAWAL_METHOD': str,
                                'WITHDRAWAL_AMOUNT': str,
                                'CUS_CUST_ID': str,
                                'CUS_FULL_NAME': str,
                                'CUS_CUST_DOC_NUMBER': str,
                                'CUS_CUST_DOC_TYPE': str,
                                'CUS_ADDRESS': str,
                                'CUS_RU_SINCE_DT': str,
                                'CUS_RU_SINCE_DT_MP': str,
                                'CUS_CUST_STATUS_MP': str,
                                'CUS_PHONE': str,
                                'CUS_COUNTRY': str,
                                'CUS_STATE': str,
                                'CUS_CITY': str,
                                'CUS_ZIP_CODE': str,
                                'CUS_CUST_ID_BUY': str,
                                'CUS_FULL_NAME_BUY': str,
                                'CUS_CUST_DOC_NUMBER_BUY': str,
                                'CUS_CUST_DOC_TYPE_BUY': str,
                                'CUS_ADDRESS_BUY': str,
                                'CUS_RU_SINCE_DT_BUY': str,
                                'CUS_RU_SINCE_DT_MP_BUY': str,
                                'CUS_CUST_STATUS_MP_BUY': str,
                                'CUS_PHONE_BUY': str,
                                'CUS_COUNTRY_BUY': str,
                                'CUS_STATE_BUY': str,
                                'UF': str,
                                'CUS_CITY_BUY': str,
                                'CUS_ZIP_CODE_BUY': str,
                                'CUS_CUST_ID_SEL': str,
                                'CUS_FULL_NAME_SEL': str,
                                'CUS_CUST_DOC_NUMBER_SEL': str,
                                'CUS_CUST_DOC_TYPE_SEL': str,
                                'CUS_ADDRESS_SEL': str,
                                'CUS_RU_SINCE_DT_SEL': str,
                                'CUS_RU_SINCE_DT_MP_SEL': str,
                                'CUS_CUST_STATUS_MP_SEL': str,
                                'CUS_PHONE_SEL': str,
                                'CUS_COUNTRY_SEL': str,
                                'CUS_STATE_SEL': str,
                                'CUS_CITY_SEL': str,
                                'CUS_ZIP_CODE_SEL': str,
                                'GENERATED_AT': str,
                                'HAD_REPORT': str})

lister_csv = lister_csv[['CUS_CUST_DOC_NUMBER', 'CUS_CUST_ID_BUY', 'CUS_FULL_NAME', 'CUS_CUST_DOC_NUMBER_SEL',
                         'CUS_CUST_ID_SEL', 'CUS_FULL_NAME_SEL', 'MOV_MOVE_ID', 'PAY_PAYMENT_ID', 'MOV_CREATED_DT',
                         'PAY_REASON_ID', 'MOV_DETAIL', 'PAY_OPERATION_TYPE_ID', 'MOV_AMOUNT', 'WIT_WITHDRAW_ID',
                         'ACCOUNT_TYPE', 'WIT_BANK_ACC_NUMBER', 'RECEIVING_ACCOUNT_HOLDER_NAME', 'RECEIVING_ACCOUNT_HOLDER_DOC_NUMBER',
                         'BANK_ID', 'WIT_BANK_ACC_BRANCH_ID', 'RECEIVING_BANK_NAME', 'OFICIO_NUMBER']]

lister_csv = lister_csv.rename(columns={'CUS_CUST_DOC_NUMBER': 'Documento Origem (Investigado)',
                                        'CUS_CUST_ID_BUY': 'ID Interno  Origem (Investigado)',
                                        'CUS_FULL_NAME': 'Nome Registro Origem (Investigado)',
                                        'CUS_CUST_DOC_NUMBER_SEL': 'Documento Destino  (Contraparte)',
                                        'CUS_CUST_ID_SEL': 'ID Interno  Destino (Contraparte)',
                                        'CUS_FULL_NAME_SEL': 'Nome Registro Destino (Contraparte)',
                                        'MOV_MOVE_ID': 'ID Movimentação Interno',
                                        'PAY_PAYMENT_ID': 'ID Pagamento Interno',
                                        'MOV_CREATED_DT': 'Data Movimentação',
                                        'PAY_REASON_ID': 'Descrição da Movimentação',
                                        'MOV_DETAIL': 'Tipo  Movimentação',
                                        'PAY_OPERATION_TYPE_ID': 'Detalhe Movimentação',
                                        'MOV_AMOUNT': 'Valor  Movimentação',
                                        'WIT_WITHDRAW_ID': 'ID Saque Interno',
                                        'ACCOUNT_TYPE': 'Tipo Conta Bancária',
                                        'WIT_BANK_ACC_NUMBER': 'Número Conta',
                                        'RECEIVING_ACCOUNT_HOLDER_NAME': 'Nome Recebedor Saque',
                                        'RECEIVING_ACCOUNT_HOLDER_DOC_NUMBER': 'Documento Recebedor',
                                        'BANK_ID': 'Banco Recebedor',
                                        'WIT_BANK_ACC_BRANCH_ID': 'Número Agência',
                                        'RECEIVING_BANK_NAME': 'Número Agência',
                                        'OFICIO_NUMBER': 'OFICIO_NUMBER'})

lister_csv.to_excel('/alloc/data/fury_oficios-quebra-sigilo/notebooks/casos.xlsx', index= False, header = True)

print(lister_df.shape, dt.datetime.now())